In [1]:
from PIL import Image
import os

In [2]:
image_dir = r"C:\Users\davide.biganzoli\OneDrive - Università degli Studi di Milano\Lavoro\2024\Methylation\207127940135"

In [3]:
# Lista dei file immagine
image_files = [file for file in os.listdir(image_dir) if file.endswith(('jpg', 'jpeg', 'png'))]

In [4]:
# Filtra le immagini per colore e per il primo campione
green_images = [file for file in image_files if 'green' in file and 'R01C01' in file]

In [5]:
def get_t_value(filename):
    # Estrai il valore di T dal nome del file
    base_name = os.path.basename(filename)
    t_index = base_name.find('_T') + 2
    t_value = int(base_name[t_index:t_index+2])  # Assumiamo che T sia seguito da due cifre
    return t_value

In [6]:
def get_r_value(filename):
    # Estrai il valore di R dal nome del file
    base_name = os.path.basename(filename)
    r_index = base_name.find('_R') + 2
    r_value = int(base_name[r_index:r_index+2])  # Assumiamo che R sia seguito da due cifre
    return r_value

In [7]:
def get_num_value(filename):
    # Estrai il valore del numero verticale dal nome del file
    base_name = os.path.basename(filename)
    num_part = base_name.split('_')[2]
    num_value = int(num_part.split('-')[0])
    return num_value

In [23]:
def create_mosaic(image_list, output_path, vertical_shift_percent=0.5, horizontal_shift_percent=0.048):
    # Ordina le immagini in base al nome per mantenere l'ordine
    image_list.sort(key=lambda x: (get_num_value(x), get_t_value(x)))

    # Apri tutte le immagini
    images = []
    for img_file in image_list:
        try:
            img = Image.open(os.path.join(image_dir, img_file))
            img.verify()  # Verifica l'integrità dell'immagine
            img = Image.open(os.path.join(image_dir, img_file))  # Ri-apri l'immagine dopo verifica
            img.load()  # Carica i dati dell'immagine
            images.append(img)
        except (IOError, SyntaxError) as e:
            print(f"Immagine {img_file} corrotta o non valida: {e}")
            continue

    # Assumi che tutte le immagini abbiano la stessa dimensione
    if not images:
        print("Nessuna immagine valida trovata.")
        return

    img_width, img_height = images[0].size
    vertical_overlap = int(img_height * vertical_shift_percent)  # Sovrapposizione verticale personalizzata
    horizontal_overlap = int(img_width * horizontal_shift_percent)  # Sovrapposizione orizzontale personalizzata

    # Determina il numero di righe e colonne
    num_rows = len(set(get_num_value(img) for img in image_list))
    num_cols = 13  # Numero di colonne per ogni shift di T

    # Crea una nuova immagine grande abbastanza per contenere tutte le altre
    mosaic_width = img_width * num_cols - horizontal_overlap * (num_cols - 1)
    mosaic_height = img_height * num_rows - vertical_overlap * (num_rows - 1)
    mosaic_image = Image.new('RGB', (mosaic_width, mosaic_height))

    # Aggiungi ogni immagine nel mosaico
    for img, file in zip(images, image_list):
        t_value = get_t_value(file) - 1  # T01 corrisponde alla colonna 0
        num_value = get_num_value(file) - 1  # Numero corrisponde allo spostamento verticale
        x = t_value * (img_width - horizontal_overlap)
        y = num_value * (img_height - vertical_overlap)  # Aggiungi sovrapposizione
        mosaic_image.paste(img, (x, y))

    # Salva il mosaico
    mosaic_image.save(output_path, format='PNG')

In [24]:
# Crea il mosaico per le immagini "green" del primo campione
create_mosaic(green_images, r"C:\Users\davide.biganzoli\OneDrive - Università degli Studi di Milano\Lavoro\2024\Methylation\green_mosaic_first_sample.png")